In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

dataF = yf.download("EURUSD=X", start="2022-11-13", end="2023-01-11", interval='5m')
dataF.iloc[:,:]

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-11-14 00:00:00,1.033058,1.033058,1.032844,1.032844,1.032844,0
2022-11-14 00:05:00,1.032951,1.033592,1.032951,1.032951,1.032951,0
2022-11-14 00:10:00,1.033271,1.033485,1.033058,1.033058,1.033058,0
2022-11-14 00:15:00,1.033378,1.033805,1.033271,1.033271,1.033271,0
2022-11-14 00:20:00,1.033271,1.033805,1.033271,1.033592,1.033592,0
...,...,...,...,...,...,...
2023-01-10 23:35:00,1.073998,1.074114,1.073998,1.074114,1.074114,0
2023-01-10 23:40:00,1.074114,1.074460,1.073998,1.074460,1.074460,0
2023-01-10 23:45:00,1.074114,1.074576,1.074114,1.074114,1.074114,0


In [31]:
grid_distance = 0.005
TPSL_Ratio = 1
midprice = 1.065

In [32]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=0.1)
grid


array([0.965, 0.97 , 0.975, 0.98 , 0.985, 0.99 , 0.995, 1.   , 1.005,
       1.01 , 1.015, 1.02 , 1.025, 1.03 , 1.035, 1.04 , 1.045, 1.05 ,
       1.055, 1.06 , 1.065, 1.07 , 1.075, 1.08 , 1.085, 1.09 , 1.095,
       1.1  , 1.105, 1.11 , 1.115, 1.12 , 1.125, 1.13 , 1.135, 1.14 ,
       1.145, 1.15 , 1.155, 1.16 , 1.165])

In [33]:
signal = [0]*len(dataF)
i=0
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High)<p and max(row.Low, row.High)>p:
            signal[i]=1
    i+=1

In [34]:
dataF["signal"]=signal
dataF[dataF["signal"]==1]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2022-11-14 01:15:00,1.034661,1.035090,1.034661,1.034875,1.034875,0,1
2022-11-14 08:15:00,1.034447,1.035197,1.034447,1.035197,1.035197,0,1
2022-11-14 08:20:00,1.035197,1.035197,1.034447,1.034447,1.034447,0,1
2022-11-14 10:15:00,1.029548,1.030078,1.029124,1.030078,1.030078,0,1
2022-11-14 10:35:00,1.030078,1.030078,1.029654,1.029654,1.029654,0,1
...,...,...,...,...,...,...,...
2023-01-10 14:25:00,1.074922,1.075153,1.074807,1.074922,1.074922,0,1
2023-01-10 14:30:00,1.075269,1.075384,1.074922,1.075269,1.075269,0,1
2023-01-10 14:40:00,1.075038,1.075038,1.074460,1.074460,1.074460,0,1


In [41]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high = dfpl.High, low = dfpl.Low, close = dfpl.Close, length = 16)
dfpl.dropna(inplace=True)

In [61]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class MyStrat(Strategy):
    mysize = 0.5
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = self.data.ATR[-1] #grid_distance
        TPSLRatio = 1.2*TPSL_Ratio

        if self.signal1==1 and len(self.trades)<=2:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/1, commission=.000)
stat = bt.run()
stat

Start                     2022-11-14 01:20:00
End                       2023-01-10 23:55:00
Duration                     57 days 22:35:00
Exposure Time [%]                   14.441371
Equity Final [$]                   100.959312
Equity Peak [$]                    101.015129
Return [%]                           0.959312
Buy & Hold Return [%]                 3.87755
Return (Ann.) [%]                    6.042815
Volatility (Ann.) [%]                1.060293
Sharpe Ratio                         5.699194
Sortino Ratio                       11.715163
Calmar Ratio                         16.92914
Max. Drawdown [%]                   -0.356948
Avg. Drawdown [%]                   -0.042216
Max. Drawdown Duration       17 days 05:30:00
Avg. Drawdown Duration        0 days 20:54:00
# Trades                                 1648
Win Rate [%]                        30.339806
Best Trade [%]                        0.31599
Worst Trade [%]                     -0.321524
Avg. Trade [%]                    

In [54]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False, 
plot_pl=False, plot_volume=False, plot_drawdown=False, smooth_equity=False, relative_equity=True, 
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)

Row(id='5787', ...)

In [28]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,16,16,1.034875,1.034875,0.000000,0.000000,2022-11-14 01:20:00,2022-11-14 01:20:00,0 days 00:00:00
1,4,100,122,1.035197,1.030197,-0.020000,-0.004830,2022-11-14 08:20:00,2022-11-14 10:10:00,0 days 01:50:00
2,189,16,123,1.034875,1.029548,-1.006845,-0.005148,2022-11-14 01:20:00,2022-11-14 10:15:00,0 days 08:55:00
3,2,124,124,1.030503,1.030503,0.000000,0.000000,2022-11-14 10:20:00,2022-11-14 10:20:00,0 days 00:00:00
4,3,128,376,1.029548,1.037154,0.022818,0.007388,2022-11-14 10:40:00,2022-11-15 07:20:00,0 days 20:40:00
...,...,...,...,...,...,...,...,...,...,...
115,188,11332,11533,1.064736,1.072463,1.452626,0.007257,2023-01-06 19:15:00,2023-01-09 13:30:00,2 days 18:15:00
116,1,11478,11559,1.069747,1.075153,0.005406,0.005053,2023-01-09 08:55:00,2023-01-09 15:40:00,0 days 06:45:00
117,1,11559,11559,1.075153,1.075153,0.000000,0.000000,2023-01-09 15:40:00,2023-01-09 15:40:00,0 days 00:00:00
118,4,11560,11946,1.074807,1.074114,-0.002771,-0.000645,2023-01-09 15:45:00,2023-01-10 23:55:00,1 days 08:10:00
